<h1 align='center'>Initialize Mission Workspace<h1>

In [ ]:
### Block 1 ### Initialize directory paths

import datetime, os, geopandas, shutil, zipfile #py7zr
## DEFINE DIRECTORIES
productionDirectory = "C:/WorkSpace/IRmapper/.Alberta_mapping/production_workspace"
gdrive = "G:/Shared drives/GIS Data Center/Delivered To The Goverment/Alberta_deliverables"
#os.chdir(f'{productionDirectory}')
## DEFINE DATES
delivery_date = datetime.date.today().strftime('%y%m%d')
print(f"COMPLETED - Map delivery day is {delivery_date}, paths imported: {'productionDirectory' in vars()}")

In [ ]:
### Block 2 ### Check not deleting previous work, setup local directories

today_folder = f'{productionDirectory}/{delivery_date}'
assert False == os.path.exists(today_folder), "processing folder for this date already exists, go delete if restarting validation"
os.makedirs(f'{today_folder}/_camera_data_{delivery_date}')
os.makedirs(f'{today_folder}/_client_request_data_{delivery_date}')
cam_file = f'{today_folder}/_camera_data_{delivery_date}'

print(f"COMPLETED - local directiories built: {os.path.exists(cam_file)}")

In [ ]:
### Block 3 ### Make Gdrive directory

month_numeric = datetime.date.today().strftime('%m')
month_literal = datetime.date.today().strftime('%B')
day_numeric = datetime.date.today().strftime('%d')
gdrive_deliverables_path = f'{gdrive}/{month_numeric}_{month_literal}/{month_literal.upper()}_{day_numeric}/'
if not os.path.exists(gdrive_deliverables_path):
    os.makedirs(gdrive_deliverables_path)
    print(f"COMPLETED - cloud folder built: {os.path.exists(gdrive_deliverables_path)}")
else:
    print(f"COMPLETED - cloud folder built: False (already existed)")

### ANALYST ACTION: paste geodata from gov request email to "_client_request_data" folder

In [ ]:
### Block 4 ### Extract zip files from gov request geodata

counter = 0
for root, dirs, files in os.walk(f'{today_folder}/_client_request_data_{delivery_date}'):
    for name in files:
        if name.endswith('.zip'):
            with zipfile.ZipFile(os.path.join(root, name), 'r') as zip:
                zip.extractall(root)
                counter += 1
                print(f'{counter}. zip file "{name}" extracted')
        #elif name.endswith('.7z'):
        #    with py7zr.SevenZipFile(os.path.join(root, name), 'r') as z:
        #        z.extractall(root)
        
if counter == 0:
    print("COMPLETED - no compressed files in request data")
else:
    print("---")
    print(f"COMPLETED - {counter} compressed file(s) extracted")

In [ ]:
### BLOCK 5 ### Request data validation system

## Validation: verify file contains column "FIRE_NUMBE"
for root, dirs, files in os.walk(f'{today_folder}/_client_request_data_{delivery_date}'):
    for name in files:        
        if name.endswith('.shp'):
            shapefile = geopandas.read_file(os.path.join(root, name))
            fire_columns = shapefile.columns
            assert "FIRE_NUMBE" in fire_columns, f"Shapefile attribute table doesn't contain field FIRE_NUMBE - file: {name} - build identification field before proceeding"
print("All files contain FIRE_NUMBE identification column")

## Validation: verify correct CRS
for root, dirs, files in os.walk(f'{today_folder}/_client_request_data_{delivery_date}'):
    for name in files:
        if name.endswith('.shp'):
            shapefile = geopandas.read_file(os.path.join(root, name))
            assert shapefile.crs == "EPSG:3400", f"CRS incorrect for this jursidiction - file name: {name} - change to EPSG:3400 before proceeding"
print("All files have correct CRS")

## Validation: verify each fire is a single multipart object - not multiple singlepart geometries
import collections
for root, dirs, files in os.walk(f'{today_folder}/_client_request_data_{delivery_date}'):
    for name in files:
        if name.endswith('.shp'):
            shapefile = geopandas.read_file(os.path.join(root, name))
            fireID_list = list(shapefile['FIRE_NUMBE'])
            duplicated = [k for k, v in collections.Counter(fireID_list).items() if v > 1]
            assert duplicated == [], f"Fire(s) {duplicated} in file {name} is in multiple singlepart features - merge into multipart before proceeding"
print("All fires are built of a single multipart geometry")

## Validation: verify no repeated fires between files
requested_fires = []
for root, dirs, files in os.walk(f'{today_folder}/_client_request_data_{delivery_date}'):
    for name in files:        
        if name.endswith('.shp'):
            shapefile = geopandas.read_file(os.path.join(root, name))
            for fire in shapefile['FIRE_NUMBE']:
                requested_fires.append(fire)
duplicated = [k for k, v in collections.Counter(requested_fires).items() if v > 1]
assert duplicated == [], f"Fires {duplicated} are duplicated across request files - delete down to one instance of fire before proceeding"
print("No fire IDs are repeated across different request files")

print("---")
print(f"COMPLETED - all validation checks were passed")

In [ ]:
### BLOCK 6 ### Unpack fires polygons from request geodata into individual fires workspaces

requested_fires = []
for root, dirs, files in os.walk(f'{today_folder}/_client_request_data_{delivery_date}'):
    for name in files:        
        if name.endswith('.shp'):
            gdf_fires = geopandas.read_file(os.path.join(root, name))
            fires_list = gdf_fires['FIRE_NUMBE'].unique()
            print(f'fires_list from {name}: {fires_list}')
            for fire in fires_list:
                requested_fires.append(fire)
                single_fire_gdf = gdf_fires[gdf_fires['FIRE_NUMBE'] == fire]
                os.makedirs(f'{today_folder}/{fire}_{delivery_date}/Scratch')
                single_fire_gdf.to_file(f'{today_folder}/{fire}_{delivery_date}/Scratch/{fire}_request_perimeter.shp')    
if len(requested_fires) == 0:
    def prRed(skk): print("\033[91m {}\033[00m" .format(skk))
    prRed(f'all fires: {requested_fires}, no fires the found in the request')
else:
    print(f'all fires: {requested_fires}')
    print("---")
    print(f"COMPLETED - {len(requested_fires)} workspace(s) built")

In [ ]:
### BLOCK 7 ### Build workspace for each requested fire + spare fire

#add spare fire workspace
if "ZZZ999" not in requested_fires:
    requested_fires.append("ZZZ999")
    os.makedirs(f'{today_folder}/ZZZ999_{delivery_date}/Scratch')
    open(f'{today_folder}/ZZZ999_{delivery_date}/Scratch/ZZZ999_request_perimeter.shp', "w+")

#Make subdirectory for each fire post-production mapping
for fire in requested_fires:
    src="C:/WorkSpace/IRmapper/application_materials_v1.0/python_scripts/AB/AB_mapping.ipynb"
    dst=f"{today_folder}/{fire}_{delivery_date}/{fire}-processing.ipynb"
    shutil.copy(src,dst)

    #make a copy of the processing code template for local fire
    src="C:/WorkSpace/IRmapper/application_materials_v1.0/QGIS_projects/AB_template.qgz"
    dst=f"{today_folder}/{fire}_{delivery_date}/{fire}_{delivery_date}.qgz"
    shutil.copy(src,dst)
    print(f"Built workspace for {fire}")
print("---")
print("COMPLETED - move on to individual fire processing scripts")